In [ ]:
!pip3 install xgboost

In [ ]:
#import dask.array as da 
import pandas as pd
#import dask.dataframe as dd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

## Loading data

In [ ]:
#df = pd.read_csv("/store/ronnieleeper/activeProjects/soilQC/SoilMoistureQC/data/acclima_soil_water-full_wo_duplicates-20221130.csv",dtype={'PRECIPITATION': 'float64'})
#new dataset
df = pd.read_csv("/store/ronnieleeper/activeProjects/soilQC/SoilMoistureQC/data/acclima_soil_water-full_with_tipping_bucket_and_wetness-20230113.csv",dtype={'PRECIPITATION': 'float64'})# Old data - /store/ronnieleeper/activeProjects/soilQC/SoilMoistureQC/data/acclima_soil_water-full_wo_duplicates-20221130.csv
# Add column to easily detect normal / flagged data
df['FLAG'] = df.NOPRCPRESPONSE + df.FROZENRECOVERY+ df.NOISE + df.FAILURE + df.STATIC+df.ERRATIC+ df.DIURNALNOISE+df.TOOHIGH+df.SCALING + df.ZERO+df.SPIKE

In [ ]:
df['UTC_START'] = pd.to_datetime(df['UTC_START'])

## Get number of unique sensors for each station

In [ ]:
#Stations and the number of sensors they have
station_sensor_counts = df.groupby('STATION_ID').VARIABLE.nunique()
station_sensor_counts

In [ ]:
station_sensor_dist = df.groupby('STATION_ID').VARIABLE.value_counts()
station_sensor_dist= pd.DataFrame(station_sensor_dist)

In [ ]:
station_with_15_vars = list(station_sensor_counts[station_sensor_counts == 15].index)
station_with_14_vars = list(station_sensor_counts[station_sensor_counts == 14].index)
station_with_13_vars = list(station_sensor_counts[station_sensor_counts == 13].index)
station_with_more_than_13_vars = list(station_sensor_counts[station_sensor_counts >= 13].index)

In [ ]:
# Just sample code for plotting available sensor readings over time
fig, ax = plt.subplots(1,1)
sns.scatterplot(data=df[(df['STATION_ID']==1012)],x='UTC_START',y='VARIABLE', ax=ax)
ax.set_title("Sensor data for station 1012")
ax.set_xlabel("Time stamps (UTC_START)")
ax.set_ylabel("Sensors (VARIABLE)- 13 present")
#ax.set_xlim(1,10)
#plt.xlim(x_limit)
plt.xticks(rotation=45)
plt.savefig('./data_plots/13sensors/station_1012_sensors.png')

## Common Preprocessing pipeline

In [ ]:
# Convert precipitation to Boolean - TEMPORARY MEASURE, CHANGE ONCE NEW DATA OBTAINED
df['PRECIPITATION'] = df['PRECIPITATION'].fillna(value= 0.0)
#df.PRECIPITATION = df.where(df.PRECIPITATION.compute() == 0, 1) # TEMPORARY MEASURE, CHANGE ONCE NEW DATA OBTAINED
def make_boolean(x):
    if x <500.0:
        return 1
    elif x > 500.0:
        return 0
#df['PRECIPITATION'] = df['PRECIPITATION'].apply(lambda x: make_boolean(x))
df['PRECIPITATION'] = df['WETNESS_1'].apply(lambda x: make_boolean(x))
print("Null values in precipitation",df['PRECIPITATION'].isnull().sum())

In [ ]:
# modifying dataset by merging output classes into noise, spike or no flag
df_modified = df.drop(['NOPRCPRESPONSE','ZERO','TOOHIGH','ERRATIC','FAILURE','SCALING','FLAG','TIPPING_BUCKET','WETNESS_2'],axis=1)
#0 -> non-flagged/ normal, 1-> SPIKE, 2 -> Noise
def update_flag(row):
    if row['SPIKE'] == 1:
        return 1
    elif row['NOISE'] == 1 or row['DIURNALNOISE'] == 1 or row['FROZENRECOVERY']==1:
        return 2
    else:
        return 0
df_modified['FLAG'] = df_modified.apply(lambda x: update_flag(x), axis=1)
df_modified.drop(['FROZENRECOVERY','DIURNALNOISE','NOISE','SPIKE'],axis=1,inplace=True)
df_modified.drop('WBAN',axis=1,inplace=True)
df_modified.drop('WETNESS_1',axis=1,inplace=True)
df_modified.drop('STATIC',axis=1,inplace=True)

In [ ]:
# Remove outliers 
df_modified = df_modified[(df_modified['VOLUMETRIC'] > 0.0) & (df['VOLUMETRIC'] < 0.6)]

In [ ]:
# Normalize data per station 
TODO: HAVE TO NORMALOSE TRAIN AND TEST SEPERATELY. IT IS DONE AT THE START OF EACH APPROACH
#df_modified[['TEMPERATURE','VOLUMETRIC']] = df_modified[['TEMPERATURE','VOLUMETRIC','STATION_ID','VARIABLE']].groupby(['STATION_ID', 'VARIABLE']).transform(lambda x: (x - x.mean()) / x.std())

In [ ]:
def extract_depth(row):
    return int(row[-3:])

df_modified['VARIABLE'] = df_modified['VARIABLE'].apply(extract_depth)

## LazyPredict : to get a sense of classifier performance. Run as script

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split

df_modified_sampled = df_modified.sample(n=100)
X = df_modified_sampled.drop(['FLAG'],axis=1)
y = df_modified_sampled['FLAG']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state =42)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)

models,predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)

models.to_csv('./data_plots/param_opt_models_sample_tab.csv',sep='\t')
predictions.to_csv('./data_plots/param_opt_predictions_sample.csv')

In [1]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split

df_modified_sampled = df_modified.sample(n=100)
X = df_modified_sampled.drop(['FLAG'],axis=1)
y = df_modified_sampled['FLAG']

x_subset = X#.sample(n=1000)  takes too long to run in notebook, so sample just to check. run on full data as script
y_subset = y#.sample(n=1000)

X_train, X_test, y_train, y_test = train_test_split(x_subset,y_subset,test_size=.2,random_state =42)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)

NameError: name 'df_modified' is not defined

## Approach 2


use the data as is to learn “rules” specific to stations at various sensor depth. The input columns now are:   {station_ID, sensor_name (depth), volumetric, temp, precip_binary, flag}

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import auc, accuracy_score, confusion_matrix
import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, confusion_matrix
from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier

In [ ]:
# NORMALIZING TRAIN AND TEST DATA SEPERATELY ON STATION AND VARIABLE
from sklearn.preprocessing import Normalizer
df_modified.drop('UTC_START',axis=1,inplace=True)

#data1 = df_modified[['VOLUMETRIC','TEMPERATURE','STATION_ID','VARIABLE']].groupby(['STATION_ID', 'VARIABLE']).transform(lambda x: (x - x.mean()) / x.std())
temp = {}
vol = {}

for g in x_train[['VOLUMETRIC','TEMPERATURE','STATION_ID','VARIABLE']].groupby(['STATION_ID', 'VARIABLE']):
    temp[g[0]] = (g[1].TEMPERATURE.mean(), g[1].TEMPERATURE.std())
    vol[g[0]] = (g[1].VOLUMETRIC.mean(), g[1].VOLUMETRIC.std())
    x_train.loc[g[1].index,'TEMPERATURE'] = (x_train.loc[g[1].index,'TEMPERATURE'] - g[1].TEMPERATURE.mean())/g[1].TEMPERATURE.std()
    x_train.loc[g[1].index,'VOLUMETRIC'] = (x_train.loc[g[1].index,'VOLUMETRIC'] - g[1].VOLUMETRIC.mean())/g[1].VOLUMETRIC.std()

for g in x_test[['VOLUMETRIC','TEMPERATURE','STATION_ID','VARIABLE']].groupby(['STATION_ID', 'VARIABLE']):
    x_test.loc[g[1].index,'TEMPERATURE'] = (x_test.loc[g[1].index,'TEMPERATURE'] - temp[g[0]][0])/temp[g[0]][1]
    x_test.loc[g[1].index,'VOLUMETRIC'] = (x_test.loc[g[1].index,'VOLUMETRIC'] - temp[g[0]][0])/temp[g[0]][1]

In [ ]:
skf = StratifiedKFold(n_splits=20,shuffle=True,random_state=42)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
        xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
        dummy_clf = DummyClassifier(strategy="most_frequent")
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        xgb_model.fit(X_train, y_train)
        dummy_clf.fit(X_train, y_train)

        print("In split: ",i)
        #models.append(xgb_model)
        y_pred = xgb_model.predict(X_test)
        y_pred_dummy = dummy_clf.predict(X_test)
        labels = xgb_model.classes_
        conf_df = pd.DataFrame(confusion_matrix(y_test, y_pred), columns=labels, index=labels)
        conf_df.index.name = 'True labels'
        print("----Confusion matrix:")
        print(conf_df)

        f_macro = f1_score(y_test, y_pred, average='macro')
        f_none = f1_score(y_test, y_pred, average=None)
        acc = accuracy_score(y_test, y_pred)
        f_macro_dummy = f1_score(y_test, y_pred_dummy, average='macro')
        f_none_dummy = f1_score(y_test, y_pred_dummy, average=None)
        acc_dummy = accuracy_score(y_test, y_pred_dummy)
        print("----F1 score macro (xgboost): " , f_macro)
        print("----F1 score none (xgboost): " , f_none)
        print("----Accuracy (xgboost): ", acc)
        print("----F1 score macro (dummy): " , f_macro_dummy)
        print("----F1 score none (dummy): " , f_none_dummy)
        print("----Accuracy (dummy): ", acc_dummy)


### sample weights for unbalanced data

In [ ]:
from sklearn.utils import class_weight
classes_weights = class_weight.compute_sample_weight(
    class_weight='balanced',
    y=y_train
)

xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
xgb_model.fit(x_train, y_train,sample_weight=classes_weights)

y_pred = xgb_model.predict(x_test)
#y_pred_dummy = dummy_clf.predict(X_test)
    
f_macro = f1_score(y_test, y_pred, average='macro')
f_none = f1_score(y_test, y_pred, average=None)
acc = accuracy_score(y_test, y_pred)
print("----F1 score macro (xgboost): " , f_macro)
print("----F1 score none (xgboost): " , f_none)
print("----Accuracy (xgboost): ", acc)
                              
labels = xgb_model.classes_
conf_df = pd.DataFrame(confusion_matrix(y_test, y_pred), columns=labels, index=labels)
conf_df.index.name = 'True labels'
print(conf_df)

### parameter optimization using RandomsearchCV

In [ ]:
# Run detailed model as script
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils import class_weight

x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=0,stratify= y)
parameters = {
      'learning_rate': [0.5,0.6, 0.65, 0.7, 0.8],
      'max_depth': [ 25,30,40,55],
      'n_estimators': [300,500,750,1250, 1500, 1750],
      "reg_alpha"   : [0.3,0.4,0.5,0.6],
      "reg_lambda"  : [2],
      "gamma"       : [0.1,0.5,1]}
xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
grid_obj_xgb = RandomizedSearchCV(xgb_model,parameters, cv=5,n_iter=40,verbose=3,n_jobs=30)
classes_weights = class_weight.compute_sample_weight(class_weight='balanced', y=y_train)
search = grid_obj_xgb.fit(x_train, y_train,sample_weight=classes_weights,verbose=4)
print("best params: ",search.best_params_)
best_model = search.best_estimator_
y_pred = best_model.predict(x_test)

f_macro = f1_score(y_test, y_pred, average='macro')
f_none = f1_score(y_test, y_pred, average=None)
acc = accuracy_score(y_test, y_pred)
print("----F1 score macro (xgboost): " , f_macro)
print("----F1 score none (xgboost): " , f_none)
print("----Accuracy (xgboost): ", acc)
                              
labels = xgb_model.classes_
conf_df = pd.DataFrame(confusion_matrix(y_test, y_pred), columns=labels, index=labels)
conf_df.index.name = 'True labels'
print(conf_df)

## Approach 1

In [ ]:
# Take only stations with 13 or more sensor readings
df_modified = df_modified[df_modified.STATION_ID.isin(station_with_more_than_13_vars)]

In [ ]:
# Pivot the data, so for a given timestamp and station there is 1 row with values for all sensor 
df_pivot = df_modified.pivot(index=['STATION_ID','UTC_START'],columns='VARIABLE',values=['TEMPERATURE','VOLUMETRIC','PRECIPITATION','FLAG'])

In [ ]:
#To convert multiple flag columns to only 1 column
df_pivot['FLAG_upd'] = df_pivot.FLAG.max(axis=1)

df_pivot['FLAG_upd'] = df_pivot['FLAG_upd'].astype(int)

In [ ]:
df_pivot.drop([('FLAG', 'sw1005'),
            (         'FLAG', 'sw1010'),
            (         'FLAG', 'sw1020'),
            (         'FLAG', 'sw1050'),
            (         'FLAG', 'sw1100'),
            (         'FLAG', 'sw2005'),
            (         'FLAG', 'sw2010'),
            (         'FLAG', 'sw2020'),
            (         'FLAG', 'sw2050'),
            (         'FLAG', 'sw2100'),
            (         'FLAG', 'sw3005'),
            (         'FLAG', 'sw3010'),
            (         'FLAG', 'sw3020'),
            (         'FLAG', 'sw3050'),
            (         'FLAG', 'sw3100')],axis=1,inplace=True)

In [ ]:
#Make station ID as a column
df_pivot.reset_index('STATION_ID', inplace=True)
df_pivot.tail()

In [ ]:
def consolidate_temp(row):
    #return max(row[('TEMPERATURE','sw1010')],row[('TEMPERATURE','sw1005')],row[('TEMPERATURE','sw1020')],row[('TEMPERATURE','sw1050')],row[('TEMPERATURE','sw1100')],row[('TEMPERATURE','sw2010')],row[('TEMPERATURE','sw2005')],row[('TEMPERATURE','sw2020')],row[('TEMPERATURE','sw2050')],row[('TEMPERATURE','sw2100')],row[('TEMPERATURE','sw3010')],row[('TEMPERATURE','sw3005')],row[('TEMPERATURE','sw3020')],row[('TEMPERATURE','sw3050')],row[('TEMPERATURE','sw3100')])
    idx = row['TEMPERATURE'].first_valid_index()
    return row[('TEMPERATURE',idx)]
def consolidate_precip(row):
    idx = row['PRECIPITATION'].first_valid_index()
    return row[('PRECIPITATION',idx)]
#    return max(row[('PRECIPITATION','sw1010')],row[('PRECIPITATION','sw1005')],row[('PRECIPITATION','sw1020')],row[('PRECIPITATION','sw1050')],row[('PRECIPITATION','sw1100')],row[('PRECIPITATION','sw2010')],row[('PRECIPITATION','sw2005')],row[('PRECIPITATION','sw2020')],row[('PRECIPITATION','sw2050')],row[('PRECIPITATION','sw2100')],row[('PRECIPITATION','sw3010')],row[('PRECIPITATION','sw3005')],row[('PRECIPITATION','sw3020')],row[('PRECIPITATION','sw3050')],row[('PRECIPITATION','sw3100')])

df_pivot['TEMPERATURE_1'] = df_pivot.apply(lambda x: consolidate_temp(x), axis=1)
df_pivot['PRECIPITATION_1'] = df_pivot.apply(lambda x: consolidate_precip(x), axis=1)

In [ ]:
def update_missing_sensor(row):
    row[('VOLUMETRIC','sw1010')] = max(row[('VOLUMETRIC','sw1010')] ,row[('VOLUMETRIC','sw2010')],row[('VOLUMETRIC','sw3010')])
    row[('VOLUMETRIC','sw1005')] = max(row[('VOLUMETRIC','sw1005')] ,row[('VOLUMETRIC','sw2005')],row[('VOLUMETRIC','sw3050')])
    row[('VOLUMETRIC','sw1020')] = max(row[('VOLUMETRIC','sw1020')] ,row[('VOLUMETRIC','sw2020')],row[('VOLUMETRIC','sw3020')])
    row[('VOLUMETRIC','sw1050')] = max(row[('VOLUMETRIC','sw1050')] ,row[('VOLUMETRIC','sw2050')],row[('VOLUMETRIC','sw3050')])
    row[('VOLUMETRIC','sw1100')] = max(row[('VOLUMETRIC','sw1100')] ,row[('VOLUMETRIC','sw2100')],row[('VOLUMETRIC','sw3100')])
    
    # For temaperature
    #row[('TEMPERATURE','sw1010')] = max(row[('TEMPERATURE','sw1010')] ,row[('TEMPERATURE','sw2010')],row[('TEMPERATURE','sw3010')])
    #row[('TEMPERATURE','sw1005')] = max(row[('TEMPERATURE','sw1005')] ,row[('TEMPERATURE','sw2005')],row[('TEMPERATURE','sw3050')])
    #row[('TEMPERATURE','sw1020')] = max(row[('TEMPERATURE','sw1020')] ,row[('TEMPERATURE','sw2020')],row[('TEMPERATURE','sw3020')])
    #row[('TEMPERATURE','sw1050')] = max(row[('TEMPERATURE','sw1050')] ,row[('TEMPERATURE','sw2050')],row[('TEMPERATURE','sw3050')])
    #row[('TEMPERATURE','sw1100')] = max(row[('TEMPERATURE','sw1100')] ,row[('TEMPERATURE','sw2100')],row[('TEMPERATURE','sw3100')])
    
    #row[('PRECIPITATION','sw1010')] = max(row[('PRECIPITATION','sw1010')] ,row[('PRECIPITATION','sw2010')],row[('PRECIPITATION','sw3010')])
    #row[('PRECIPITATION','sw1005')] = max(row[('PRECIPITATION','sw1005')] ,row[('PRECIPITATION','sw2005')],row[('PRECIPITATION','sw3050')])
    #row[('PRECIPITATION','sw1020')] = max(row[('PRECIPITATION','sw1020')] ,row[('PRECIPITATION','sw2020')],row[('PRECIPITATION','sw3020')])
    #row[('PRECIPITATION','sw1050')] = max(row[('PRECIPITATION','sw1050')] ,row[('PRECIPITATION','sw2050')],row[('PRECIPITATION','sw3050')])
    #row[('PRECIPITATION','sw1100')] = max(row[('PRECIPITATION','sw1100')] ,row[('PRECIPITATION','sw2100')],row[('PRECIPITATION','sw3100')])
    
    return row
df_pivot = df_pivot.apply(lambda x: update_missing_sensor(x), axis=1)

In [ ]:
df_pivot.drop([
            (         'TEMPERATURE', 'sw1005'),
            (         'TEMPERATURE', 'sw1010'),
            (         'TEMPERATURE', 'sw1020'),
            (         'TEMPERATURE', 'sw1050'),
            (         'TEMPERATURE', 'sw1100'),
            (         'TEMPERATURE', 'sw2005'),
            (         'TEMPERATURE', 'sw2010'),
            (         'TEMPERATURE', 'sw2020'),
            (         'TEMPERATURE', 'sw2050'),
            (         'TEMPERATURE', 'sw2100'),
            (         'TEMPERATURE', 'sw3005'),
            (         'TEMPERATURE', 'sw3010'),
            (         'TEMPERATURE', 'sw3020'),
            (         'TEMPERATURE', 'sw3050'),
            (         'TEMPERATURE', 'sw3100')],axis=1,inplace=True)

df_pivot.drop([
            (         'VOLUMETRIC', 'sw2005'),
            (         'VOLUMETRIC', 'sw2010'),
            (         'VOLUMETRIC', 'sw2020'),
            (         'VOLUMETRIC', 'sw2050'),
            (         'VOLUMETRIC', 'sw2100'),
            (         'VOLUMETRIC', 'sw3005'),
            (         'VOLUMETRIC', 'sw3010'),
            (         'VOLUMETRIC', 'sw3020'),
            (         'VOLUMETRIC', 'sw3050'),
            (         'VOLUMETRIC', 'sw3100')],axis=1,inplace=True)

df_pivot.drop([
            (         'PRECIPITATION', 'sw1005'),
            (         'PRECIPITATION', 'sw1010'),
            (         'PRECIPITATION', 'sw1020'),
            (         'PRECIPITATION', 'sw1050'),
            (         'PRECIPITATION', 'sw1100'),
            (         'PRECIPITATION', 'sw2005'),
            (         'PRECIPITATION', 'sw2010'),
            (         'PRECIPITATION', 'sw2020'),
            (         'PRECIPITATION', 'sw2050'),
            (         'PRECIPITATION', 'sw2100'),
            (         'PRECIPITATION', 'sw3005'),
            (         'PRECIPITATION', 'sw3010'),
            (         'PRECIPITATION', 'sw3020'),
            (         'PRECIPITATION', 'sw3050'),
            (         'PRECIPITATION', 'sw3100')],axis=1,inplace=True)

In [ ]:
df_pivot.dropna(axis=0,inplace=True)
df_pivot.reset_index(inplace=True)

In [ ]:
X = df_pivot.drop([(    'UTC_START',       ''),(     'FLAG_upd',       '')],axis=1)
y = df_pivot['FLAG_upd']

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, accuracy_score, confusion_matrix

xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
xgb_model.fit(x_train, y_train)
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=0,stratify= df_pivot['FLAG_upd'])  
y_pred = xgb_model.predict(x_test)

labels = xgb_model.classes_
conf_df = pd.DataFrame(confusion_matrix(y_test, y_pred), columns=labels, index=labels)
conf_df.index.name = 'True labels'
conf_df

### K fold CV

In [ ]:
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, confusion_matrix

xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
accuracies =[]

skf = StratifiedKFold(n_splits=11)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    #train = df.loc[train_index,:]
    #test = df.loc[test_index,:]
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    xgb_model.fit(X_train, y_train)
    y_pred = xgb_model.predict(X_test)
    accuracies.append(accuracy_score(y_test, y_pred))
accuracies

### Group K Fold

In [ ]:
from sklearn.model_selection import GroupKFold

accuracies =[]
xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
X = df_pivot.drop([(    'UTC_START',       ''),(     'FLAG_upd',       '')],axis=1)
y = df_pivot['FLAG_upd']
skf = GroupKFold(n_splits=len(df_pivot.groupby('STATION_ID')))
groups = df_pivot['STATION_ID'].values
for i, (train_index, test_index) in enumerate(skf.split(X, y,groups)):
    #train = df.loc[train_index,:]
    #test = df.loc[test_index,:]
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    xgb_model.fit(X_train, y_train)
    y_pred = xgb_model.predict(X_test)
    accuracies.append(accuracy_score(y_test, y_pred))

### Stratified group K fold

In [ ]:
from sklearn.model_selection import StratifiedGroupKFold

accuracies =[]
xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
X = df_pivot.drop([(    'UTC_START',       ''),(     'FLAG_upd',       '')],axis=1)
y = df_pivot['FLAG_upd']
skf = StratifiedGroupKFold(n_splits=len(df_pivot.groupby('STATION_ID')))
groups = df_pivot['STATION_ID'].values
for i, (train_index, test_index) in enumerate(skf.split(X, y,groups)):
    #train = df.loc[train_index,:]
    #test = df.loc[test_index,:]
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    xgb_model.fit(X_train, y_train)
    y_pred = xgb_model.predict(X_test)
    accuracies.append(accuracy_score(y_test, y_pred))

## Unsupervised learning - Isolation Forest

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest

In [ ]:
#sample only noise and normal data
df_modified = df_modified[(df_modified['FLAG'] == 2) | (df_modified['FLAG'] == 0)]

X = df_modified.drop(['FLAG','UTC_START'],axis=1)
y = df_modified['FLAG']

x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=0,stratify= y)   
# To find contamination for IF
len(df_modified[df_modified['FLAG'] == 2])/len(df_modified['FLAG'])

In [ ]:
X_modified2 = X.drop('PRECIPITATION', axis=1)

random_state = np.random.RandomState(42)
model2=IsolationForest(n_estimators=300,max_samples='auto',contamination=float(0.04),random_state=random_state)
model2.fit(X_modified2)

In [ ]:
X_modified2['anomaly_scores'] = model2.predict(X_modified2)
#X_modified2['scores'] = model2.decision_function(X_modified2.drop('anomaly_scores',axis=1))
X_modified2['TRUE_VAL'] = y

In [ ]:
def get_conf_matrix_vals(row):
    if row['anomaly_scores'] == -1 and row['TRUE_VAL'] == 2:
        return 0 #TP
    elif row['anomaly_scores'] == -1 and row['TRUE_VAL'] == 0:
        return 1 #FP
    elif row['anomaly_scores'] == 1 and row['TRUE_VAL'] == 0:
        return 2 #TN
    else:
        return 3 #FN
X_modified2['conv_matrix_value'] = X_modified2.apply(lambda x: get_conf_matrix_vals(x), axis=1)